In [1]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core

!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 5.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15837, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 15837 (delta 30), reused 12 (delta 12), pack-reused 15771 (from 2)
Receiving objects: 100% (15837/15837), 6.40 MiB | 23.16 MiB/s, done.
Resolving deltas: 100% (11532/11532), done.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your e

In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.5 ; cuda:  cu124
detectron2: 0.6


In [3]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [34]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/drive/MyDrive/ESC499/Slur_Detection_Augm/data/train/train.json", "/content/drive/MyDrive/ESC499/Slur_Detection_Augm/data/train")
register_coco_instances("my_dataset_val", {}, "/content/drive/MyDrive/ESC499/Slur_Detection_Augm/data/val/val.json", "/content/drive/MyDrive/ESC499/Slur_Detection_Augm/data/val")

train_metadata = MetadataCatalog.get("my_dataset_train")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train")
val_metadata = MetadataCatalog.get("my_dataset_val")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val")

[02/18 03:06:44 d2.data.datasets.coco]: Loaded 48 images in COCO format from /content/drive/MyDrive/ESC499/Slur_Detection_Augm/data/train/train.json
[02/18 03:06:44 d2.data.datasets.coco]: Loaded 9 images in COCO format from /content/drive/MyDrive/ESC499/Slur_Detection_Augm/data/val/val.json


In [35]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

class TrainerWithEvaluator(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Defines the evaluator for validation.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)


cfg = get_cfg()
cfg.OUTPUT_DIR = "/content/drive/MyDrive/ESC499/Slur_Detection_Augm/models"
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.TEST.EVAL_PERIOD = 500
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0002
cfg.SOLVER.MAX_ITER = 4000
cfg.SOLVER.STEPS = [3200, 3600]
cfg.SOLVER.GAMMA = 0.3
# cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = TrainerWithEvaluator(cfg)         # Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False)        # Load a pretrained model if available (resume training) or start training from scratch if no pretrained model is available


[02/18 03:07:27 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [36]:
trainer.train()

[02/18 03:07:40 d2.engine.train_loop]: Starting training from iteration 0
[02/18 03:08:00 d2.utils.events]:  eta: 1:05:17  iter: 19  total_loss: 5.813  loss_cls: 0.7852  loss_box_reg: 0.09612  loss_mask: 0.6953  loss_rpn_cls: 3.58  loss_rpn_loc: 0.6263    time: 0.9795  last_time: 0.9873  data_time: 0.8069  last_data_time: 0.7812   lr: 3.9962e-06  max_mem: 9213M
[02/18 03:08:20 d2.utils.events]:  eta: 1:04:40  iter: 39  total_loss: 2.754  loss_cls: 0.7599  loss_box_reg: 0.07131  loss_mask: 0.6879  loss_rpn_cls: 0.6293  loss_rpn_loc: 0.5844    time: 0.9775  last_time: 1.0224  data_time: 0.7707  last_data_time: 0.8050   lr: 7.9922e-06  max_mem: 9213M
[02/18 03:08:39 d2.utils.events]:  eta: 1:04:15  iter: 59  total_loss: 2.41  loss_cls: 0.6607  loss_box_reg: 0.09394  loss_mask: 0.6733  loss_rpn_cls: 0.3822  loss_rpn_loc: 0.526    time: 0.9710  last_time: 0.9807  data_time: 0.7512  last_data_time: 0.7758   lr: 1.1988e-05  max_mem: 9213M
[02/18 03:08:59 d2.utils.events]:  eta: 1:04:00  iter:

In [15]:
import yaml

config_yaml_path = "/content/drive/MyDrive/ESC499/Slur_Detection_Augm/models/config.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

In [37]:
# Inference should use the config with parameters that are used in training
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

[02/18 04:18:34 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/ESC499/Slur_Detection_Augm/models/model_final.pth ...


/usr/local/lib/python3.11/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

In [39]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader


register_coco_instances("my_dataset_val2", {}, "/content/drive/MyDrive/ESC499/Slur_Detection_Augm/data/val2/val.json", "/content/drive/MyDrive/ESC499/Slur_Detection_Augm/data/val2")
val_metadata = MetadataCatalog.get("my_dataset_val2")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val2")


evaluator = COCOEvaluator("my_dataset_val2", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val2")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[02/18 04:25:47 d2.data.datasets.coco]: Loaded 31 images in COCO format from /content/drive/MyDrive/ESC499/Slur_Detection_Augm/data/val2/val.json
[02/18 04:25:47 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
[02/18 04:25:47 d2.data.datasets.coco]: Loaded 31 images in COCO format from /content/drive/MyDrive/ESC499/Slur_Detection_Augm/data/val2/val.json
[02/18 04:25:47 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[02/18 04:25:47 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[02/18 04:25:47 d2.data.common]: Serializing 31 elements to byte tensors and concatenating them all ...
[02/18 04:25:47 d2.data.common]: Serialized dataset takes 1.16 MiB
[02/18 04:25:47 d2.evaluation.evaluator]: Start inference on 31 batches
[02/18 04:26:33 d2.evaluation.evaluator]: Inference don